In [2]:
from transformers import pipeline, set_seed
import pandas as pd
import requests, sys, webbrowser,xml
import numpy as np
import bs4

In [3]:
generator = pipeline('text-generation', model='gpt2')

C:\Users\Jesse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\transformers\modeling_auto.py:796: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Given a number letters and some sequence, returns the sequence with generated attached words
def generateText(generator,intro_sequence:str, num_words = 5)->str:
    # Imported random text generation
    text = generator(intro_sequence, max_length=len(intro_sequence) + num_words, num_return_sequences=1)[0].get("generated_text")
    return endSentence(text)
def endSentence(text):
    last_char = text[-1]
    #if((last_char != '?') and (last_char != '!') and (last_char != '.' )):
    return text

In [4]:
def getText(url = "https://www.keepinspiring.me/famous-quotes/" ):
    # Requesting data from url, finding specialized tags for this particular website
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, "html.parser")
    text  = soup.find_all("div", class_ = 'author-quotes')  
    return text
text = getText()

## Game List: 
1) **Mad Libs** 

2) **Find the real quote**

3) **How well do you know your favorite song?**
    

In [5]:
#Tag processing functions to clean up nasty html formatting, replaces div tags
def processing_div(tag):
    return tag.replace('<div class="author-quotes">', "").replace("</div>", "")

#span processing, replaces span tag
def processing_span(tag):
    return tag.replace("<span class=\"quote-author-name\">", "").replace("</span>", "")

# Checks for tags that have yet to be removed, not given standard format. Reasoning - we don't know when ads will pop up
def cleaner(table):
    arr = np.array([])
    for i in table.get("quote"):
        arr = np.append(arr,("<" in i))
    clean = table[(arr != 1)]
    return clean

In [8]:
table = pd.DataFrame().assign(quote = text)

#Formatting, processing, and splitting quotes and authors
def tableProcess(table):
    table = table.assign(quote = table.get("quote").apply(str))
    table = table.assign(quote = table.get("quote").apply(processing_div).apply(processing_span))
    table = cleaner(table)
    table = table.assign(author  = table.get("quote").apply(authors), quote = table.get("quote").apply(removeAuthors))
    return table
table = tableProcess(table)
table

,quote,author
0,“You know you’re in love when you can’t fall a...,– Dr. Suess
1,"“I’m selfish, impatient and a little insecure....",– Marilyn Monroe
2,“Get busy living or get busy dying.”,– Stephen King
3,“The first step toward success is taken when y...,– Mark Caine
5,“Twenty years from now you will be more disapp...,– Mark Twain
6,“When I dare to be powerful – to use my streng...,– Audre Lorde
8,“A successful man is one who can lay a firm fo...,– David Brinkley
10,“I can’t give you a sure-fire formula for succ...,-Herbert Bayard Swope
11,“Would you like me to give you a formula for s...,– Thomas J. Watson
12,"“It is hard to fail, but it is worse never to ...",– Theodore Roosevelt


In [7]:
# Obtainin author from quote
def authors(quote):
    return quote.split("”")[1]

#Removing author and adding lost smartquote
def removeAuthors(quote):
    return (quote.split("”")[0] + ("”"))

In [6]:
import random
def replace_words_at_random(generator, word_arr: [str], num_words: int):
    # Pick num_words indices out of word_arr
    # Store the chosen indices
    indices = np.array([])
    for i in range(num_words):
        # Pick the next random index
        index = random.randint(3,len(word_arr)-1)

        # Store the index of the next chosen word
        indices = np.append(indices, index)
        join  = " ".join(word_arr[:index])
        # Generate the next word
        text = generateText(generator,join, 10)
        word_arr[index] =  text.split(" ")[index]
    indices = np.unique(indices)
    return word_arr, indices
stringo = "Wake me up inside Call my name and save me from the dark Bid my blood to run Before I come undone"
words, indices = replace_words_at_random(generator,stringo.split(" "),12)
print(words)
indices

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['Wake', 'me', 'up', 'inside', 'she', '–', 'name', 'and', 'address', 'you', 'Go', 'on', 'dark', 'clouds', 'my', 'girl', 'to', 'my', 'Before', 'you', 'are', 'undone']


array([ 4.,  5.,  8.,  9., 10., 11., 13., 15., 17., 19., 20.])

In [7]:
song = """Never gonna give you up Never gonna let you down Never gonna run around and desert you Never gonna make you cry Never gonna say goodbye Never gonna tell a lie and hurt you""".split(" ")
words = replace_words_at_random(generator, song, 12)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [8]:
print(words)

(['Never', 'gonna', 'give', 'an', 'up', "you're", 'old,', 'let', 'the', 'down', 'Never', 'gonna', 'run', 'away', 'Never', 'desert', 'you', 'Never', 'gonna', 'die', 'any', 'less', 'Never', 'gonna', 'say', 'goodbye', 'Never', 'gonna', 'tell', 'a', 'lie', 'and', 'pretend', 'not'], array([ 3.,  5.,  6.,  8., 13., 14., 18., 19., 20., 21., 32., 33.]))
